# Evolving a Lunar Lander with differentiable Genetic Programming

## Installation
To install the required libraries run the command:

In [1]:
#!pip install -r requirements.txt

## Imports
Imports from the standard genepro-multi library are done here. Any adjustments (e.g. different operators) should be made in the notebook. For example:

```
class SmoothOperator(Node):
  def __init__(self):
    super(SmoothOperator,self).__init__()
    self.arity = 1
    self.symb = "SmoothOperator"

  def _get_args_repr(self, args):
    return self._get_typical_repr(args,'before')

  def get_output(self, X):
    c_outs = self._get_child_outputs(X)
    return np.smoothOperation(c_outs[0])

  def get_output_pt(self, X):
    c_outs = self._get_child_outputs_pt(X)
    return torch.smoothOperation(c_outs[0])
```

In [2]:
import gymnasium as gym

from genepro.node_impl import *
from genepro.evo import Evolution
from genepro.node_impl import Constant

import torch
import torch.optim as optim

import random
import os
import copy
from collections import namedtuple, deque

import matplotlib.pyplot as plt
from matplotlib import animation

## Reinforcement Learning Setup
Here we first setup the Gymnasium environment. Please see https://gymnasium.farama.org/environments/box2d/lunar_lander/ for more information on the environment. 

Then a memory buffer is made. This is a buffer in which state transitions are stored. When the buffer reaches its maximum capacity old transitions are replaced by new ones.

A frame buffer is initialised used to later store animation frames of the environment.

In [3]:
env = gym.make("LunarLander-v2", render_mode="rgb_array")

In [4]:
Transition = namedtuple("Transition", ("state", "action", "next_state", "reward"))


class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

    def __iadd__(self, other):
        self.memory += other.memory
        return self

    def __add__(self, other):
        self.memory = self.memory + other.memory
        return self

## Fitness Function

Here you get to be creative. The default setup evaluates 5 episodes of 300 frames. Think of what action to pick and what fitness function to use. The Multi-tree takes an input of $n \times d$ where $n$ is a batch of size 1.

In [5]:
def fitness_function_pt(multitree, num_episodes=5, episode_duration=300, ignore_done=False, render=False):
    memory = ReplayMemory(10000)
    episode_rewards = []
    if render:
        frames = []

    # print(multitree.get_readable_repr())
    for _ in range(num_episodes):
        # get initial state of the environment
        observation = env.reset()
        observation = observation[0]
        rewards = []
        for _ in range(episode_duration):
            if render:
                frames.append(env.render())
            input_sample = torch.from_numpy(observation.reshape((1, -1))).float()
            action = torch.argmax(multitree.get_output_pt(input_sample)).detach()
            observation, reward, terminated, truncated, info = env.step(action.item())
            rewards.append(reward)
            output_sample = torch.from_numpy(observation.reshape((1, -1))).float()
            memory.push(input_sample, torch.tensor([[action.item()]]), output_sample, torch.tensor([reward]))
            if (terminated or truncated) and not ignore_done:
                break
        episode_rewards.append(np.sum(rewards))

    # Get the average reward over all episodes
    fitness = episode_rewards
    if render:
        return fitness, memory, frames
    return fitness, memory

def fitness_function(multitree, num_episodes=5, episode_duration=300, ignore_done=False, render=False):
    env = gym.make("LunarLander-v2", render_mode="rgb_array")


    episode_rewards = []
    if render:
        frames = []

    for _ in range(num_episodes):
        # get initial state of the environment
        observation = env.reset()
        observation = observation[0]
        rewards = []
        for _ in range(episode_duration):
            if render:
                frames.append(env.render())
            input_sample = observation.reshape((1, -1))
            action = np.argmax(multitree.get_output(input_sample))
            observation, reward, terminated, truncated, info = env.step(action.item())
            rewards.append(reward)
            if (terminated or truncated) and not ignore_done:
                break
        episode_rewards.append(np.sum(rewards))

    # Get the average reward over all episodes
    fitness = np.array(episode_rewards)
    if render:
        return fitness, [], frames
    return fitness, []

In [6]:
### USED TO STORE THE EXPERIMENT DICTIONARY
import inspect
import itertools
import pickle


def serialize_functions_in_dict(dictionary):
    for key, value in dictionary.items():
        if inspect.isfunction(value) or inspect.ismethod(value):
            dictionary[key] = value.__name__
        elif isinstance(value, list):
            for i, item in enumerate(value):
                if isinstance(item, dict):
                    value[i] = serialize_functions_in_dict(item)
                elif inspect.isfunction(item) or inspect.ismethod(item):
                    value[i] = item.__name__
                elif isinstance(item, Node):
                    value[i] = item.symb
        elif isinstance(value, dict):
            dictionary[key] = serialize_functions_in_dict(value)
    return dictionary

### USED TO CREATE THE EXPERIMENT DICTIONARY
def grid_search_params(params_dict):
    """
    Given a dictionary of hyperparameters, if a value is a list, loop over all values
    and create a grid search.
    """
    param_keys = params_dict.keys()
    param_values = params_dict.values()
    param_combinations = list(itertools.product(*[v if isinstance(v, list) else [v] for v in param_values]))
    for combination in param_combinations:
        yield dict(zip(param_keys, combination))

In [7]:
# Save the gen as a pickle file in the gens folder
def save_and_evaluate_evo_generations(evo, fitness_function, experiment_name, num_episodes=10):
    generation_evo_fitnesses = []
    generation_test_fitnesses = []
    for i, gen in enumerate(evo.best_of_gens):
        if i == 0:
            continue

        episode_rewards, _ = fitness_function(gen, num_episodes=num_episodes)
        evo_fitness_mean, evo_fitness_std = round(np.mean(gen.fitnesses), 3), round(np.std(gen.fitnesses), 3)
        test_fitness_mean, test_fitness_std  = round(np.mean(episode_rewards), 3), round(np.std(episode_rewards), 3)
        print(f"Best of Generation {i}: evo fitness:{evo_fitness_mean}+/-{evo_fitness_std} \t test_fitness:{test_fitness_mean}+/-{test_fitness_std}")
        
        generation_evo_fitnesses.append(gen.fitnesses)
        generation_test_fitnesses.append(episode_rewards)
        # create the gens folder if it doesn't exist
        os.makedirs(f"./experiments/{experiment_name}/gen/", exist_ok=True) 
        with open(f"./experiments/{experiment_name}/gen/gen_{i}_{evo_fitness_mean}_{test_fitness_mean}.pickle", "wb") as f:
            pickle.dump(gen, f)

    np.save(f"./experiments/{experiment_name}/generation_evo_fitnesses.npy", generation_evo_fitnesses)
    np.save(f"./experiments/{experiment_name}/generation_test_fitnesses.npy", generation_test_fitnesses)   
    return generation_evo_fitnesses, generation_test_fitnesses

def plot_evo_test_fitnesses(evo_fitnesses, test_fitnesses, experiment_name):
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_title(f"Fitnesses: {experiment_name}")
    ax.set_xlabel("Generation")
    ax.set_ylabel("Fitness")
    ax.plot(np.arange(len(evo_fitnesses)), [np.mean(gen) for gen in evo_fitnesses], label="evo_fitness", color='tab:blue')
    ax.fill_between(np.arange(len(evo_fitnesses)), [np.mean(gen) - np.std(gen) for gen in evo_fitnesses], [np.mean(gen) + np.std(gen) for gen in evo_fitnesses], alpha=0.2, color='tab:blue')
    ax.plot(np.arange(len(test_fitnesses)), [np.mean(gen) for gen in test_fitnesses], label="test_fitness", color='tab:orange')
    ax.fill_between(np.arange(len(test_fitnesses)), [np.mean(gen) - np.std(gen) for gen in test_fitnesses], [np.mean(gen) + np.std(gen) for gen in test_fitnesses], alpha=0.2, color='tab:orange')
    ax.legend()
    plt.savefig(f"./experiments/{experiment_name}/{experiment_name}.png")
    plt.close()

## Evolution Setup
Here the leaf and internal nodes are defined. Think about the odds of sampling a constant in this default configurations. Also think about any operators that could be useful and add them here. 

Adjust the population size (multiple of 8 if you want to use the standard tournament selection), max generations and max tree size to taste. Be aware that each of these settings can increase the runtime.

#### BASELINE

In [ ]:
from copy import deepcopy
import json
from genepro.selection import elitism_selection, tournament_selection
from genepro.variation import coeff_mutation, subtree_crossover, subtree_mutation

experiment_name = "fitness_mean-len-sqrt_std"
num_features = env.observation_space.shape[0]
evo_settings = {
    "fitness_function": fitness_function,
    "internal_nodes": [[Plus(), Minus(), Times(), Div(), Sin(), Cos(), Log(), Sqrt(), Square(), Max(), Min()]],
    "leaf_nodes": [[Feature(i) for i in range(num_features)] + [Constant()]],
    "n_trees": 4,
    "pop_size": 32,
    "max_gens": 500,
    "init_max_depth": 4,
    "max_tree_size": 256,
    "crossovers": [[{"fun": subtree_crossover, "rate": 0.5}]],
    "mutations": [[{"fun": subtree_mutation, "rate": 0.5}]],
    "coeff_opts": [[{"fun": coeff_mutation, "rate": 0.5}]],
    "selection": {"fun": tournament_selection, "kwargs": {"tournament_size": 4}},
    # "selection": {"fun": elitism_selection},
    "n_jobs": 8,
    "verbose": True
}

def hpo_evolve(evo_settings, experiment_name):
    hpo_settings = list(grid_search_params(evo_settings))
    for settings in hpo_settings:
        serialized_dict = serialize_functions_in_dict(deepcopy(settings))
        print(serialized_dict)
        
    for i, settings in enumerate(hpo_settings):
        specific_experiment_name = experiment_name + f"_pops{settings['pop_size']}_gens{settings['max_gens']}_mts{settings['max_tree_size']}_cor{settings['crossovers'][0]['rate']}_mutr{settings['mutations'][0]['rate']}_coeffr{settings['coeff_opts'][0]['rate']}"
        os.makedirs(f"./experiments/{specific_experiment_name}", exist_ok=True)
        with open(f"./experiments/{specific_experiment_name}/evo_settings.json", "w") as f:
            serialized_dict = serialize_functions_in_dict(deepcopy(settings))
            json.dump(serialized_dict, f)

        evo_baseline = Evolution(**settings)
        evo_baseline.evolve()

        with open(f"./experiments/{specific_experiment_name}/evolution_class.pickle", "wb") as f:
            pickle.dump(evo_baseline, f)
        
        generation_evo_fitnesses, generation_test_fitnesses = save_and_evaluate_evo_generations(evo_baseline, fitness_function, specific_experiment_name, num_episodes=5)
        plot_evo_test_fitnesses(generation_evo_fitnesses, generation_test_fitnesses, specific_experiment_name)
        
hpo_evolve(evo_settings, experiment_name=experiment_name)

{'fitness_function': 'fitness_function', 'internal_nodes': ['+', '-', '*', '/', 'sin', 'cos', 'log', 'sqrt', '**2', 'max', 'min'], 'leaf_nodes': ['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'const?'], 'n_trees': 4, 'pop_size': 32, 'max_gens': 500, 'init_max_depth': 4, 'max_tree_size': 256, 'crossovers': [{'fun': 'subtree_crossover', 'rate': 0.5}], 'mutations': [{'fun': 'subtree_mutation', 'rate': 0.5}], 'coeff_opts': [{'fun': 'coeff_mutation', 'rate': 0.5}], 'selection': {'fun': 'tournament_selection', 'kwargs': {'tournament_size': 4}}, 'n_jobs': 8, 'verbose': True}
gen: 1 (4.5s),	best of gen fitness: -143.371; reward:-123.845+/-20.483,	best of gen size: 15
gen: 2 (5.7s),	best of gen fitness: -117.521; reward:-103.162+/-40.447,	best of gen size: 8
gen: 3 (7.1s),	best of gen fitness: -123.813; reward:-99.631+/-38.213,	best of gen size: 18
gen: 4 (8.3s),	best of gen fitness: -110.032; reward:-86.608+/-55.109,	best of gen size: 16
gen: 5 (9.6s),	best of gen fitness: -87.310; r

### Run multiple experiments and plot

In [ ]:
# NUM_EXPERIMENTS = 2
# SAVE_DIR = "./plots/plot_data"
# experiment_name = "plot_test"

# evo_settings = {
#     "fitness_function": fitness_function_pt,
#     "internal_nodes": [Plus(), Minus(), Times(), Div()],
#     "leaf_nodes": [Feature(i) for i in range(num_features)] + [Constant()],
#     "n_trees": 4,
#     "pop_size": 8,
#     "max_gens": 10,
#     "init_max_depth": 4,
#     "max_tree_size": 32,
#     "crossovers": [{"fun": subtree_crossover, "rate": 0.5}],
#     "mutations": [{"fun": subtree_mutation, "rate": 0.5}],
#     "coeff_opts": [{"fun": coeff_mutation, "rate": 0.5}],
#     "selection": {"fun": tournament_selection, "kwargs": {"tournament_size": 8}},
#     "n_jobs": 8,
#     "verbose": True
# }

# os.makedirs(f"./experiments/{experiment_name}", exist_ok=True)
# with open(f"./experiments/{experiment_name}/evo_settings.json", "w") as f:
#     serialized_dict = serialize_functions_in_dict(deepcopy(evo_settings))
#     json.dump(serialized_dict, f)

# def run_multiple_evolutions(num_experiments, save_path):
#     fitnesses = []
    
#     for _ in range(num_experiments):
#         evo_baseline = Evolution(**evo_settings)
#         evo_baseline.evolve()
#         fitness = save_and_evaluate_evo_generations(evo_baseline, fitness_function_pt)
#         fitnesses.append(fitness)
    
#     np.save(save_path, fitnesses)

# save_path = f"{SAVE_DIR}/{experiment_name}.npy"

# run_multiple_evolutions(NUM_EXPERIMENTS, save_path)

## Evolve
Running this cell will use all the settings above as parameters

# Test

## Make an animation
Here the best evolved individual is selected and one episode is rendered. Make sure to save your lunar landers over time to track progress and make comparisons.

In [ ]:
# # gist to save gif from https://gist.github.com/botforge/64cbb71780e6208172bbf03cd9293553
# def save_frames_as_gif(frames, path="./", filename="evolved_lander.gif"):
#     plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi=72)
#     patch = plt.imshow(frames[0])
#     plt.axis("off")

#     def animate(i):
#         patch.set_data(frames[i])

#     anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)
#     anim.save(path + filename, writer="imagemagick", fps=60)


# frames = []
# avg_fitness, frames = get_test_score(evo.best_of_gens[-1], num_episodes=5, episode_duration=300, seed=5, render=True)
# print("Average fitness of the render is: ", avg_fitness)
# env.close()
# save_frames_as_gif(frames)

## Play animation

<img src="evolved_lander.gif" width="750">

## Optimisation
The coefficients in the multi-tree aren't optimised. Here Q-learning (taken from https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) is used to optimise the weights further. Incorporate coefficient optimisation in training your agent(s). Coefficient Optimisation can be expensive. Think about how often you want to optimise, when, which individuals etc.

In [ ]:
# batch_size = 128
# GAMMA = 0.99

# constants = best.get_subtrees_consts()

# if len(constants) > 0:
#     optimizer = optim.AdamW(constants, lr=1e-3, amsgrad=True)

# for _ in range(500):
#     if len(constants) > 0 and len(evo.memory) > batch_size:
#         target_tree = copy.deepcopy(best)

#         transitions = evo.memory.sample(batch_size)
#         batch = Transition(*zip(*transitions))

#         non_final_mask = torch.tensor(
#             tuple(map(lambda s: s is not None, batch.next_state)), dtype=torch.bool
#         )

#         non_final_next_states = torch.cat(
#             [s for s in batch.next_state if s is not None]
#         )
#         state_batch = torch.cat(batch.state)
#         action_batch = torch.cat(batch.action)
#         reward_batch = torch.cat(batch.reward)

#         state_action_values = best.get_output_pt(state_batch).gather(1, action_batch)
#         next_state_values = torch.zeros(batch_size, dtype=torch.float)
#         with torch.no_grad():
#             next_state_values[non_final_mask] = (
#                 target_tree.get_output_pt(non_final_next_states).max(1)[0].float()
#             )

#         expected_state_action_values = (next_state_values * GAMMA) + reward_batch

#         criterion = nn.SmoothL1Loss()
#         loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

#         # Optimize the model
#         optimizer.zero_grad()
#         loss.backward()
#         torch.nn.utils.clip_grad_value_(constants, 100)
#         optimizer.step()

# print(best.get_readable_repr())
# print(get_test_score(best))

In [ ]:
# frames = []
# fitness_function_pt(
#     best, num_episodes=1, episode_duration=500, render=True, ignore_done=False
# )
# env.close()
# save_frames_as_gif(frames, filename="evolved_lander_RL.gif")

<img src="evolved_lander_RL.gif" width="750">